# TP 3 - Stockage des mots de passe

In [ ]:
from OutilsCrypto import *

## 1. Hachage

Une <b>fonction de hachage cryptographique</b> est une fonction qui, à une donnée de taille arbitraire, associe une image de taille fixe, et dont une propriété essentielle est qu'elle est pratiquement impossible à inverser, c'est-à-dire que si l'image d'une donnée par la fonction se calcule très efficacement, le calcul inverse d'une donnée d'entrée ayant pour image une certaine valeur se révèle impossible sur le plan pratique. Pour cette raison, on dit d'une telle fonction qu'elle est à sens unique.

Une fonction de hachage cryptographique idéale possède les propriétés suivantes :

- la fonction est déterministe, c'est-à-dire qu'un même message aura toujours la même valeur de hachage ;
- la valeur de hachage d'un message se calcule « facilement » ;
- il est impossible, pour une valeur de hachage donnée, de construire un message ayant cette valeur (résistance à la préimage) ;
- il est impossible de trouver un second message ayant la même valeur de hachage qu'un message donné (résistance à la seconde préimage) ;
- il est impossible de trouver deux messages différents ayant la même valeur de hachage (résistance aux collisions) ;
- modifier un tant soit peu un message modifie considérablement la valeur de hachage.

### 1.1 Une première fonction de hachage ?

Commençons par étudier la fonction suivante, qui consiste très simplement à ne conserver que les 8 premiers caractères de l'entrée.

In [ ]:
def hachageBidon(x):
    return x[0:8]

In [ ]:
hachageBidon("motdepasse")

<b>Exercice 1.</b> Parmi les propriétés qu'une bonne fonction de hachage doit posséder, indiquer si oui ou non la fonction <code>hachageBidon</code> convient. Vous justifierez en donnant des exemples si nécessaire.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : </b>
</div>

### 1.2 Nouvelle tentative de fonction de hachage

Essayons de faire un peu mieux. Nous allons commencer par écrire la représentation binaire (via la table ASCII) de chaque caractère du texte en entrée (on notera $l$ sa taille), puis nous allons définir la fonction de hachage suivante :
$$
h(x) = \sum_{i=0}^{l-1} x[i] \times 2^{l-i-1} \pmod{251}
$$
Ainsi, par exemple, pour un caractère "a", h("a") correspond à sa valeur décimale dans la table ASCII.

Voici une fonction qui permet de transformer une chaîne de caractères en sa représentation binaire via la table ASCII.

In [ ]:
def toBinary(a):
  return ''.join(format(ord(x), '08b') for x in a)

bin = toBinary("motdepasse")
print(bin)

<b> Exercice 2.</b> Écrire la fonction <code>hachage(x)</code> qui calculer la fonction de hachage $h(x)$ d'une chaîne de caractère <code>x</code>.

In [ ]:
def hachage(x):
    # à compléter


Utiliser le bloc suivant pour faire des tests avec différents mots.

In [ ]:
print(hachage("motdepasse")) #le résultat attendu est 206

Parmi les propriétés qu'une bonne fonction de hachage doit posséder, indiquer si oui ou non la fonction <code>hachage</code> convient. Vous justifierez en donnant des exemples si nécessaire.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : </b>
</div>

### 1.3 Des exemples plus sérieux

Ces exemples trop simples montrent qu'il est compliqué de trouver une fonction cumulant toutes ces propriétés. Le bloc suivant montre l'utilisation de l'algorithme sha-256.

In [ ]:
import hashlib as hash
texte = "test"
print("Le hachage de '",texte, "' par l'algorithme sha-256 est :")
print(hash.sha256(texte.encode('utf-8')).hexdigest())

<b>Exercice 3.</b> Parmi les propriétés qu'une bonne fonction de hachage doit posséder, indiquer si oui ou non la fonction de hachage <code>sha256</code> <b>semble</b> convenir. Vous justifierez en donnant des exemples si nécessaire.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : </b>
</div>

## 2. Chiffrement de mot de passe



Dans le contexte du stockage de comptes utilisateur dans une base de données, il est impératif de comprendre qu'on ne doit surtout pas stocker les mots de passe des utilisateurs, et encore moins en clair !

On pourrait les chiffrer à l'aide d'un des algorithmes vus précédemment, mais cette solution est à bannir, car il suffit de connaître la clé de chiffrement pour pouvoir déchiffrer l'ensembe des mots de passe. Un autre problème serait que le chiffrement/déchiffrement à l'aide d'algorithmes plus évolués a un coût en ressources non négligeable.

Une meilleure méthode est de stocker, non pas les mots de passe directement, mais le résultat de leur hachage. Ainsi, à la connexion, il suffit de comparer le hachage du mot de passe entré par l'utilisateur au hachage stocké dans la base de données. Le calcul du hachage étant rapide et la connaissance du haché d'un mot de passe ne permettant a priori pas de retrouver celui-ci, cette méthode est à privilégier pour le stockage des mots de passe.

Attention cependant, il y a des faiblesses que nous allons étudier, et des astuces pour renforcer encore la sécurité de ce type de données.

### 2.1 Première version

Le bloc de code suivant simule le stockage du hash d'un mot de passe dans une bdd puis la connexion d'un utilisateur.

In [ ]:
import hashlib as hash

## enregistrement du hash du mot de passe

password = str(input("mot de passe : "))
hashed_password = hash.sha256 (password.encode ('utf-8')).hexdigest()
print ("ce qui est stocké en bdd : ",hashed_password)

## authentification

check = str(input("essai de mot de passe : "))
hashed_check = hash.sha256(check.encode('utf-8')).hexdigest()
print ("le hash de cet essai est : ",hashed_check)
if hashed_check == hashed_password:
    print("login success")
else:
    print("login failed")


### 2.2 Attaque par dictionnaire

En cas d'intrusion dans la bdd, un utilisateur mal intentionné pourrait récupérer tous les hash des mots de passe. Bien que sha-256 soit résistant au calcul de la préimage, que pensez-vous de la sécurité de l'utilisateur qui aurait choisi comme mot de passe le mot "chaton" par exemple ?

<b>Exercice 4.</b> Compléter la fonction <code>attaque_dico_hash(h,dic)</code> qui prend en paramètres un hash <code>h</code> et un dictionnaire <code>dic</code> et qui compare <code>h</code> aux hash de chaque mot du dictionnaire. Si une correspondance est trouvée, vous afficherez le mot du dictionnaire qui a comme hash <code>h</code>, sinon vous afficherez un message disant qu'aucune correspondance n'a été trouvée.

Vous indiquerez également <b>combien de mots</b> ont été testés ainsi que le <b>temps</b> mis par l'algorithme.

In [ ]:
from datetime import datetime

def attaque_dico_hash(h, dic):
    dico = open(dic, mode="r")
    n = 0   # pour compter le nombre de mots
    t0 = datetime.now()	# l'heure à l'instant présent

    for mot in dico:
        mot = mot.strip()

        # à compléter


    print("{} mot(s) ont étés testés en {} seconde(s),".format(n, (datetime.now()-t0).total_seconds()))


Tester votre fonction à l'aide du bloc suivant. Vous prendrez le soin auparavant de créer sur votre espace personnel sur le serveur Salazar un répertoire nommé <b>Dictionnaires</b> et d'y placer le fichier <b>DictionnaireFR.txt</b> que vous récupérerez sur UMTICE et qui contient 336531 mots de la langue française.

In [ ]:
attaque_dico_hash('71ea4dee4c387f40197e36666c87e07890650161455f2932a40fbeec47673cd7','Dictionnaires/DictionnaireFR.txt')

### 2.3 Attaque par Rainbow Table

Cette attaque ne détectera que les mots de passe qui sont des mots présents dans le dictionnaire mais pas les juxtapositions de mots ou les mots de passe courants mais absents du citionnaire (motdepasse, 1234, etc.)

Dans la pratique, il existe des listes de hash de mots de passe couramment utilisés, et il est très facile de retrouver une grosse quantité de mots de passe à partir d'une base contenant des hash.

<b>Exercice 5.</b> Pour vous en convaincre, utilisez le site https://www.dcode.fr/fonction-hash pour trouver les mots de passe correpsondant aux hash suivants :

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>967520ae23e8ee14888bae72809031b98398ae4a636773e18fff917d77679334 : </b>
<b>f2d81a260dea8a100dd517984e53c56a7523d96942a834b9cdc249bd4e8c7aa9 : </b>
<b>8d969eef6ecad3c29a3a629280e686cf0c3f5d5a86aff3ca12020c923adc6c92 : </b>
</div>

### 2.4 Avec un grain de sel

Pour remédier à cette faiblesse, on va ajouter une chaîne aléatoire (qu'on appellera du <b>sel</b>) à la fin du mot de passe avant de calculer son hash, et on stockera dans la bdd le hash + le sel. Lors de l'authentification, il suffira d'ajouter le sel au mot entré par l'utilisateur puis de vérifier que le hash correspond bien à celui qui est stocké dans la base de données.

L'avantage de cette méthode est que si plusieurs utilisateurs ont le même mot de passe, le sel généré étant différent, le hash stocké sera différent ! Une attaque par Rainbow Table n'est alors plus envisageable cet il faudrait pouvoir stocker et tester les hash des mots de passe fréquents concaténés avec tous les sels possibles...

<b>Exercice 6.</b> Écrire la fonction <code>secure_password_generate(password)</code> qui a pour paramètre une chaîne de caractère <code>password</code>. Cette fonction doit :
- générer un <code>sel</code> aléatoire = une chaîne de caractères (lettres de a à z) aléatoires de longueur 32
- calculer le <code>hash</code> de la chaîne de caractère formée en concaténant <code>password</code> et <code>sel</code>
- renvoyer la chapine de caractère <code>"hash_sel"</code>

<i>Exemple :</i> secure_password_generate("motdepasse") peut renvoyer une chaîne du style 98cad44dc9a866146e7b6b402e2578e0bff796db8d3d8944b7d775ac9e23c090_yrvenpqfmcgcltpqkheyiqqnoviyyqqb
dans laquelle le sel aléatoire généré est yrvenpqfmcgcltpqkheyiqqnoviyyqqb
et le hash de la chaîne "motdepasseyrvenpqfmcgcltpqkheyiqqnoviyyqqb" est 98cad44dc9a866146e7b6b402e2578e0bff796db8d3d8944b7d775ac9e23c090

In [ ]:
import random
import string

## enregistrement du hash du mot de passe salé

def secure_password_generate(password):

    # à compléter



Tester votre fonction à l'aide du bloc suivant qui simule l'enregistrement d'un mot de passe puis une tentative de connexion.

In [ ]:
## authentification

def secure_password_validate(password, secure_password):
    secure_password_salted = secure_password.split("_")[0]
    salt =  secure_password.split("_")[1]
    submitted_password_salted = hash.sha256((password + salt).encode('utf-8')).hexdigest()
    if submitted_password_salted == secure_password_salted:
        print("login success")
    else:
        print("login failed")

password = str(input("mot de passe : "))
secured_password = secure_password_generate(password)
print ("ce qui est stocké en bdd : ",secured_password)

check = str(input("essai de mot de passe : "))
secure_password_validate(check, secured_password)

### 2.5 Utilisation de bcrypt pour le chiffrement de mot de passe

Dans la pratique, le plus simple est d'utiliser la librairie bcrypt qui permet de faire du hâché avec du sel.

Les blocs suivants illustrent l'utilisation de cette librairie pour stocker les mots de passe et pour authentifier un utilisateur.

In [ ]:
import bcrypt

password = str(input("mot de passe : ")).encode('utf-8')

salt = bcrypt.gensalt()

hashedPassword = bcrypt.hashpw(password, salt)

print("le sel :",salt)
print("ce qui est stocké en base de données : ",hashedPassword)


In [ ]:
import bcrypt

password = str(input("mot de passe : ")).encode('utf-8')

hashed = bcrypt.hashpw(password, bcrypt.gensalt(10))

print ("ce qui est stocké en bdd : ",hashed)

essai = str(input("essai de mot de passe :")).encode('utf-8')

if bcrypt.checkpw(essai, hashed) :
    print("login success")
else :
    print("login failed")

<b>Exercice 7. (Bonus)</b> S'il vous reste du temps, faîtes une petite recherche sur internet pour voir comment ajouter du poivre à ce hâché pour améliorer encore plus la sécurité !